In [ ]:
import pandas as pd
import time
from tqdm.notebook import tqdm
import datetime
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from pandas.plotting import scatter_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler

## レースID作成

In [ ]:
race_id_list = []
for year in range(2019,2020,1):
    for place in range(1, 11, 1):
        for kai in range(1, 6, 1):
            for day in range(1, 9, 1):
                for r in range(1, 13, 1):
                    race_id = (
                        str(year)
                        + str(place).zfill(2)
                        + str(kai).zfill(2)
                        + str(day).zfill(2)
                        + str(r).zfill(2)
                    )
                    race_id_list.append(race_id)

## レース結果取得

In [ ]:
def scrape_race_results(race_id_list, pre_race_results={}):
    race_results = pre_race_results
    for race_id in tqdm(race_id_list):
        if race_id in race_results.keys():
            continue
        try:
            url = "https://db.netkeiba.com/race/" + race_id
            df = pd.read_html(url)[0]
            
            if len(df) < 3:
                continue
            # horse_idとjockey_idをスクレイピング
            html = requests.get(url)
            html.encoding = "EUC-JP"
            soup = BeautifulSoup(html.text, "html.parser")
            # horse_id
            horse_id_list = []
            horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                "a", attrs={"href": re.compile("^/horse")}
            )

            for a in horse_a_list:
                horse_id = re.findall(r"\d+", a["href"])
                horse_id_list.append(horse_id[0])
            # jockey_id
            jockey_id_list = []
            jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                "a", attrs={"href": re.compile("^/jockey")}
            )
            for a in jockey_a_list:
                jockey_id = re.findall(r"\d+", a["href"])
                jockey_id_list.append(jockey_id[0])

            df["horse_id"] = horse_id_list
            df["jockey_id"] = jockey_id_list
            
            # change コースid追加
            df['course_id'] = [int(race_id[4:6])]*len(horse_id_list)
            
            race_results[race_id] = df
            time.sleep(0.1)
        except IndexError:
            continue
        except Exception as e:
            print(e)
            break
    return race_results

In [ ]:
results = scrape_race_results(race_id_list[:3])
race_id_list = list(set(list(results)))

In [ ]:
for key in results:
    results[key].index = [key] * len(results[key])

In [ ]:
results = pd.concat([results[key] for key in results], sort=False)
results.to_pickle('results.pickle')

In [ ]:
results = pd.read_pickle('results.pickle')

## レース情報取得

In [ ]:
def scrape_race_info(race_id_list):
    race_infos = {}
    for race_id in tqdm(race_id_list):
        try:
            url = "https://db.netkeiba.com/race/" + race_id
            
            df = pd.read_html(url)[0]
            # change データ取得失敗対応
            if len(df) < 3:
                continue
                
            html = requests.get(url)
            html.encoding = "EUC-JP"
            soup = BeautifulSoup(html.text, "html.parser")

            texts = (
                soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
            )
            info = re.findall(r'\w+', texts)
            info_dict = {}
            for text in info:
                if text in ["芝", "ダート"]:
                    info_dict["race_type"] = text
                if "障" in text:
                    info_dict["race_type"] = "障害"
                if "m" in text:
                    info_dict["course_len"] = int(re.findall(r"\d+", text)[0])
                if text in ["良", "稍重", "重", "不良"]:
                    info_dict["ground_state"] = text
                if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                    info_dict["weather"] = text
                if "年" in text and "月" in text and "日" in text: 
                    info_dict["date"] = text
                # change コース特性追加
                if "右" in text:
                    info_dict["course_type"] = "right"
                if "左" in text:
                    info_dict["course_type"] = "left"
                if "直線" in text:
                    info_dict["course_type"] = "straight"
            race_infos[race_id] = info_dict
            time.sleep(0.1)
        except IndexError:
            continue
        except Exception as e:
            print(e)
            break
    return race_infos

In [ ]:
race_infos = scrape_race_info(race_id_list)
df_race_infos = pd.DataFrame(race_infos.values(), index=race_infos.keys())

In [ ]:
results_addinfo = results.merge(df_race_infos,left_index=True,right_index=True,how='inner')
results_addinfo['date'] = pd.to_datetime(results_addinfo['date'],format='%Y年%m月%d日')
results_addinfo.to_pickle('results_addinfo.pickle')

In [ ]:
results_addinfo = pd.read_pickle('results_addinfo.pickle')

## 馬戦績取得

In [ ]:
horse_id_list = results_addinfo['horse_id'].unique()

In [ ]:
def scrape_horse_results(horse_id_list, pre_horse_id=[]):
    horse_results = {}
    for horse_id in tqdm(horse_id_list):
        if horse_id in pre_horse_id:
            continue
        try:
            url = 'https://db.netkeiba.com/horse/' + horse_id
            html = requests.get(url)
            html.encoding = "EUC-JP"
            soup = BeautifulSoup(html.text, "html.parser")
            
            ## add(生産地)
            texts = soup.find("div", attrs={"class": "db_prof_area_02"}).find_all("a")
            for text in texts:
                if "breeder" in str(text):
                    Borned_place = str(text)[str(text).find('e="')+3:str(text).find('">')]
            
            df = pd.read_html(url)[3]
            if df.columns[0]=='受賞歴':
                df = pd.read_html(url)[4]
                
            df["Borned_place"] = Borned_place
            horse_results[horse_id] = df
            time.sleep(0.1)
        except IndexError:
            continue
        except Exception as e:
            import traceback
            traceback.print_exc()
            print(e)
            break
        except:
            break
    return horse_results

In [ ]:
horse_results = scrape_horse_results(horse_id_list)
for key in horse_results:
    horse_results[key].index = [key] * len(horse_results[key])
df = pd.concat([horse_results[key] for key in horse_results])
df.to_pickle('horse_results.pickle')

## 馬生産地追加

In [ ]:
borned_place_list = []
for i in range(len(results_addinfo)):
    borned_place_list.append(list(set(list(horse_results[results_addinfo['horse_id'][i]]["Borned_place"])))[0])

results_addinfo["Borned_place"] = borned_place_list
results_addinfo.to_pickle('results_addinfo.pickle')

## 未実装項目

In [ ]:
df = pd.read_pickle('horse_results.pickle')
results_addinfo = pd.read_pickle('results_addinfo.pickle')

## エラー回避のために実施

In [ ]:
results_addinfo = results_addinfo[~(results_addinfo['着順'].astype(str).str.contains('\D'))]

## 詳細戦績取得

In [ ]:
class HorseResults:
    def __init__(self, horse_results):
        self.horse_results = horse_results[['日付','着順', '賞金']]
        self.preprocessing()

    def preprocessing(self):
        df = self.horse_results.copy()

        # 着順に数字以外の文字列が含まれているものを取り除く
        df['着順'] = pd.to_numeric(df['着順'], errors='coerce')
        df.dropna(subset=['着順'], inplace=True)
        df['着順'] = df['着順'].astype(int)
        df['着順'].fillna(0, inplace=True)

        df["date"] = pd.to_datetime(df["日付"])
        df.drop(['日付'], axis=1, inplace=True)

        #賞金のNaNを0で埋める
        df['賞金'].fillna(0, inplace=True)

        self.horse_results = df

    def average(self, horse_id_list, date, n_samples='all'):
        self.horse_results.reindex(horse_id_list, axis=1)
        target_df = self.horse_results.loc[horse_id_list]

        #過去何走分取り出すか指定
        if n_samples == 'all':
            filtered_df = target_df[target_df['date'] < date]
        elif n_samples > 0:
            filtered_df = target_df[target_df['date'] < date].sort_values('date', ascending=False).groupby(level=0).head(n_samples)
        else:
            raise Exception('n_samples must be >0')

        average = filtered_df.groupby(level=0)[['着順', '賞金']].mean()
        return average.rename(columns={'着順':'着順_{}R'.format(n_samples), '賞金':'賞金_{}R'.format(n_samples)})
    # change 馬の最高賞金追加
    def max_money(self, horse_id_list, date, n_samples='all'):
        self.horse_results.reindex(horse_id_list, axis=1)
        target_df = self.horse_results.loc[horse_id_list]
        
        #過去何走分取り出すか指定
        if n_samples == 'all':
            filtered_df = target_df[target_df['date'] < date]
        elif n_samples > 0:
            filtered_df = target_df[target_df['date'] < date].sort_values('date', ascending=False).groupby(level=0).head(n_samples)
        else:
            raise Exception('n_samples must be >0')
            
        max_money = filtered_df.groupby(level=0)[['賞金']].max()
        return max_money.rename(columns={'賞金':'最高賞金_{}R'.format(n_samples)})

    def merge(self, results, date, n_samples='all'):
        df = results[results['date']==date]
        horse_id_list = df['horse_id']
        merged_df = df.merge(self.average(horse_id_list, date, n_samples), left_on='horse_id',\
                             right_index=True, how='left').merge(self.max_money(horse_id_list, date, 'all'), left_on='horse_id',\
                             right_index=True, how='left')
        return merged_df

    def merge_all(self, results, n_samples='all'):
        date_list = results['date'].unique()
        merged_df = pd.concat([self.merge(results, date, n_samples) for date in tqdm(date_list)])
        return merged_df

In [ ]:
hr = HorseResults(df)
results_5R = hr.merge_all(results_addinfo, n_samples=5)
results_5R.to_pickle('results_5R.pickle')

## 血統データ取得

In [ ]:
def scrape_peds(horse_id_list, pre_peds={}):
    peds = pre_peds
    for horse_id in tqdm(horse_id_list):
        if horse_id in peds.keys():
            continue
        try:
            url = "https://db.netkeiba.com/horse/ped/" + horse_id
            df = pd.read_html(url)[0]

            generations = {}
            for i in reversed(range(5)):
                generations[i] = df[i]
                df.drop([i], axis=1, inplace=True)
                df = df.drop_duplicates()

            ped = pd.concat([generations[i] for i in range(5)]).rename(horse_id)
            peds[horse_id] = ped.reset_index(drop=True)
            time.sleep(0.1)
        except IndexError:
            continue
        except Exception as e:
            print(e)
            break
    return peds

def process_categorical(df, target_columns):
    df2 = df.copy()
    for column in target_columns:
        df2[column] = LabelEncoder().fit_transform(df2[column].fillna('Na'))
    
    #target_columns以外にカテゴリ変数があれば、ダミー変数にする
    df2 = pd.get_dummies(df2)

    for column in target_columns:
        df2[column] = df2[column].astype('category')

    return df2

def add_blood_data(horse_id_list,df):
    peds = scrape_peds(horse_id_list)
    peds = pd.concat([peds[horse_id] for horse_id in peds], axis=1).T
    peds = peds.add_prefix('peds_')
    categorical_columns = ['horse_id'] + list(peds.columns)
    df = df.merge(peds,left_on='horse_id', right_index=True, how='left')
    df = process_categorical(df, categorical_columns)
    df.drop(['horse_id'],axis = 1,inplace = True)
    return df

## 前処理項目

In [ ]:
def preprocessing_last(results):
    df = results.copy()
    
    df = df[~(df['着順'].astype(str).str.contains('\D'))]
    df['着順'] = df['着順'].astype(int)
    df['性'] = df['性齢'].map(lambda x:str(x)[0])
    
    # chaneg 馬の所属追加
    df['所属'] = df['調教師'].map(lambda x:str(x)[1])
    df['年齢'] = df['性齢'].map(lambda x:str(x)[1:]).astype(int)
    df['体重'] = df['馬体重'].str.split('(',expand = True)[0].astype(int)
    df['体重変化'] = df['馬体重'].str.split('(',expand = True)[1].str[:-1]
    
    # change 体重変化をint型へ
    object_to_int = [int(s) for s in list(df['体重変化'])]
    df['体重変化'] = object_to_int
    df['単勝'] = df['単勝'].astype(float)
    
    # change 計不があるレースは消去
    drop_lines = list(df.query('馬体重 == "計不"').index)
    df = df.drop(index=drop_lines)
    
    ## 未実装項目
    le = LabelEncoder()
    le = le.fit(df['Borned_place'])
    df['Borned_place'] = le.transform(df['Borned_place'])
    
    df.drop(['タイム'],axis=1,inplace=True)
    df.drop(['着差'],axis=1,inplace=True)
    df.drop(['調教師'],axis=1,inplace=True)
    df.drop(['性齢'],axis=1,inplace=True)
    df.drop(['馬体重'],axis=1,inplace=True)
    df.drop(['馬名'],axis=1,inplace=True)
    df.drop(['騎手'],axis=1,inplace=True)
    df.drop(['単勝'],axis=1,inplace=True)
    df.drop(['人気'],axis=1,inplace=True)
    #df.drop(['horse_id'],axis=1,inplace=True)
    df.drop(['jockey_id'],axis=1,inplace=True)
    df['rank'] = df['着順'].map(lambda x: x if x < 4 else 4)
    return df.fillna(0)
    
    return df

## 学習とテストに分割

In [ ]:
def split_data(df,test_size):
    sorted_id_list = df.sort_values('date').index.unique()
    train_id_list = sorted_id_list[:round(len(sorted_id_list)*(1-test_size))]
    test_id_list = sorted_id_list[round(len(sorted_id_list)*(1-test_size)):]
    train = df.loc[train_id_list]
    test = df.loc[test_id_list]
    return train,test

In [ ]:
results_5R = preprocessing_last(results_5R)

In [ ]:
results_5R.to_pickle('results_5R.pickle')

In [ ]:
results_5R = pd.read_pickle('results_5R.pickle')

In [ ]:
add_blood = add_blood_data(horse_id_list,results_5R)
add_blood.to_pickle('add_blood.pickle')

In [ ]:
add_blood = pd.read_pickle('add_blood.pickle')

In [ ]:
train,test = split_data(add_blood,0.3)

In [ ]:
rank_1 = train['rank'].value_counts()[1]
rank_2 = train['rank'].value_counts()[2]
rank_3 = train['rank'].value_counts()[3]

rus = RandomUnderSampler(sampling_strategy={1:rank_1,2:rank_2,3:rank_3,4:rank_1},random_state=71)

X_train = train.drop(['着順','date','rank'],axis=1)
y_train = train['rank']
X_test = test.drop(['着順','date','rank'],axis=1)
y_test = test['rank']

#X_train_rus,y_train_rus = rus.fit_sample(X_train,y_train)

## 勾配ブースティング決定木

In [ ]:
import lightgbm as lgb

params = {
    "num_leaves": 2,
    "n_estimators": 80,
    "class_weight": "balanced",
    "random_state": 100,
}

lgb_clf = lgb.LGBMClassifier(**params)
lgb_clf.fit(X_train.values,y_train.values)

print(lgb_clf.score(X_train,y_train),lgb_clf.score(X_test,y_test))

In [ ]:
importances = pd.DataFrame(
    {"features": X_train.columns, "importance": lgb_clf.feature_importances_}
)
importances.sort_values("importance", ascending=False)[:20]

In [ ]:
import pickle
pickle.dump(lgb_clf, open('lightgbm.pickle', 'wb'))

loaded_model = pickle.load(open('lightgbm.pickle', 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

## 回収率計算

In [ ]:
import pandas as pd
import time
from tqdm.notebook import tqdm
from urllib.request import urlopen

def scrape_return_tables(race_id_list, pre_return_tables={}):
    return_tables = pre_return_tables
    for race_id in tqdm(race_id_list):
        if race_id in return_tables.keys():
            continue
        try:
            url = "https://db.netkeiba.com/race/" + race_id
            f = urlopen(url)
            html = f.read()
            html = html.replace(b'<br />', b'br')
            dfs = pd.read_html(html)
            return_tables[race_id] = pd.concat([dfs[1], dfs[2]])
            time.sleep(1)
        except IndexError:
            continue
        except:
            break
    return return_tables

return_tables = scrape_return_tables(race_id_list)
for key in return_tables:
    return_tables[key].index = [key] * len(return_tables[key])
return_tables = pd.concat([return_tables[key] for key in return_tables])

In [ ]:
class Return:
    def __init__(self, return_tables):
        self.return_tables = return_tables

    @property
    def fukusho(self):
        fukusho = self.return_tables[self.return_tables[0]=='複勝'][[1,2]]
        wins = fukusho[1].str.split('br', expand=True)#.drop([3], axis=1)
        wins.columns = ['win_0', 'win_1', 'win_2']

        returns = fukusho[2].str.split('br', expand=True)#.drop([3], axis=1)
        returns.columns = ['return_0', 'return_1', 'return_2']

        df = pd.concat([wins, returns], axis=1)
        for column in df.columns:
            df[column] = df[column].str.replace(',', '')
        return df.fillna(0).astype(int)



class ModelEvaluator:
    def __init__(self, model, return_tables):
        self.model = model
        self.fukusho = Return(return_tables).fukusho

    def predict_proba(self, X):
        return self.model.predict_proba(X)[:, 1]

    def predict(self, X, threshold=0.5):
        y_pred = self.predict_proba(X)
        return [0 if p<threshold else 1 for p in y_pred]

    def score(self, y_true, X):
        return roc_auc_score(y_true, self.predict_proba(X))

    def feature_importance(self, X, n_display=20):
        importances = pd.DataFrame({"features": X.columns, 
                                    "importance": self.model.feature_importances_})
        return importances.sort_values("importance", ascending=False)[:n_display]

    def pred_table(self, X, threshold=0.5, bet_only=True):
        pred_table = X.copy()[['馬番']]
        pred_table['pred'] = self.predict(X, threshold)
        if bet_only:
            return pred_table[pred_table['pred']==1]['馬番']
        else:
            return pred_table

    def calculate_return(self, X, threshold=0.5):
        pred_table = self.pred_table(X, threshold)
        money = -100 * len(pred_table)
        df = self.fukusho.copy()
        df = df.merge(pred_table, left_index=True, right_index=True, how='right')
        for i in range(3):
            money += df[df['win_{}'.format(i)]==df['馬番']]['return_{}'.format(i)].sum()
        return money

In [ ]:
me = ModelEvaluator(lgb_clf, return_tables)

gain = {}
n_samples = 100
for i in tqdm(range(n_samples)):
    threshold = i / n_samples
    gain[threshold] = me.calculate_return(X_test, threshold)
pd.Series(gain).plot()

In [ ]:
## グラフの見方
## 左に行けば行くほど全部の馬にかけるということ